# テスト Mass-Damper-Spring

# 1. MDSPhysicalObject
# 1.1 MDSPhysicalObject 作成

In [ ]:
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)

pprint.pprint(type(flow.plant))
pprint.pprint(type(flow.plant.physical_obj))

## 1.2 MDSPhysicalObject 属性確認

In [ ]:
pprint.pprint(flow.plant.physical_obj.get_config())
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")

## 1.3 MDSPhysicalObject 属性変更

In [ ]:
flow.plant.physical_obj.mass = 3.0
flow.plant.physical_obj.damper=1.5
flow.plant.physical_obj.spring=2.5
flow.plant.physical_obj.spring_balance_pos=0.1
flow.plant.physical_obj.static_friction_coeff=0.21
flow.plant.physical_obj.dynamic_friction_coeff=0.11
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")

# 2. プラント微分方程式 解曲線の確認（初期値問題を解く）

## 2.1 モーションフローによる解

In [ ]:
#【初期値問題】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# デフォルトのプロファイル(指令速度0, 指令位置0)をロード
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=0)
# デフォルトのコントローラをロード(推力0)
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=0)
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)
# 各コンポーネントの型を確認
pprint.pprint(type(flow.discrete_time))
pprint.pprint(type(flow.mprof))
pprint.pprint(type(flow.controller))
pprint.pprint(type(flow.plant))
pprint.pprint(type(flow.plant.physical_obj))
# 物理オブジェクトのパラメータを確認
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")
wn_Hz, cc, zeta, wd_Hz = flow.plant.physical_obj.calc_char_values()
pprint.pprint(f"{wn_Hz=} [Hz]")
pprint.pprint(f"{cc=} [Ns/m]")
pprint.pprint(f"{zeta=} [-]")
pprint.pprint(f"{wd_Hz=} [Hz]")
# 物理オブジェクトの状態を確認
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")
# 物理オブジェクトの初期位置を変える
flow.plant.physical_obj.set_state(acc=0, vel=0, pos=0.1)
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")


In [ ]:
#【初期値問題】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【初期値問題】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

## 2.2 数値ソルバーによる解

### 2.2.1 座標系
* 水平方向 $x$ [m]

### 2.2.2 パラメータ
* 経過時間 $t$ [s]
* 質量 $m$ = 2 [kg]
* ダンパ係数 $c$ = 3 [Ns/m]
* ばね定数 $k$ = 80 [N/m]
* 初期位置 $x_0$ = 0.1 [m]

### 2.2.3 運動方程式

外力 = 慣性力 + ダンパ力 + ばね力 より

$$
f = m \ddot{x} + c \dot {x} + kx \qquad (式1)
$$

###  2.2.4 数値ソルバー
scipy.integrate.solve_ivp

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html

### 2.2.5 数値ソルバーへの入力準備：1階の連立微分方程式に変形

状態ベクトルを定義する。
$$
\bm{y} = \begin{pmatrix} y_0 \\ y_1 \end{pmatrix} = \begin{pmatrix} x \\ \dot{x} \end{pmatrix} \qquad (式2)
$$

状態ベクトルの時間微分
$$
\bm{\dot{y}} = \begin{pmatrix} \dot{y_0} \\ \dot{y_1} \end{pmatrix} = \begin{pmatrix} \dot{x} \\ \ddot{x} \end{pmatrix} \qquad (式3)
$$

位置の時間微分 = 速度より
$$
\dot{y_0} = \dot{x} = y_1 \qquad (式4)
$$

速度の時間微分 = 加速度。式(1)より
$$
\dot{y_1} = \ddot{x} = \frac{f}{m} - \frac{c}{m} \dot{x} - \frac{k}{m} x = \frac{f}{m} - \frac{c}{m} y_1 - \frac{k}{m} y_0 \qquad (式5)
$$

連立微分方程式（状態ベクトルの時間微分）
$$
\bm{\dot{y}} = \begin{pmatrix} \dot{y_0} \\ \dot{y_1} \end{pmatrix} = \begin{pmatrix} y_1 \\ \frac{f}{m} - \frac{c}{m} y_1 - \frac{k}{m} y_0 \end{pmatrix} \qquad (式6)
$$


### 2.2.6 数値ソルバーの実行

In [ ]:
#【初期値問題】数値ソルバーによる解析
import numpy as np
from scipy.integrate import solve_ivp

# パラメータ
m = flow.plant.physical_obj.mass  # 質量 m [kg]
c = flow.plant.physical_obj.damper  # ダンパ係数 c [Ns/m]
k = flow.plant.physical_obj.spring  # ばね定数 k [N/m]
f = 0.0  # 外力0 [N]
print(f"{m=}[kg], {c=}[Ns/m], {k=}[N/m], {f=}[N]")

# 微分方程式の定義
def diff_eq(t, y):
    # 状態ベクトルをアンパック
    x, dot_x = y
    # 位置y0の時間微分 
    dy0_dt = dot_x
    # 速度y1の時間微分 
    dy1_dt = (f - c * dot_x - k * x) / m
    return [dy0_dt, dy1_dt]
# 時間範囲 [s]
t_span = (0, 6)
# 初期状態 [位置 [m], 速度 [m/s]]
x0 = 0.1  # [m]
dot_x0 = 0.0  # [m/s]
y0 = [x0, dot_x0]
# 評価時間点
t_eval = np.arange(t_span[0], t_span[1], 0.1)
# 数値ソルバーの実行
sol = solve_ivp(diff_eq, t_span, y0, t_eval=t_eval)
# 結果の確認
print(f"{sol.success=}")
print(f"{sol.status=}")
print(f"{sol.message=}")
print(f"{sol.t=}")
print(f"{sol.y=}")

### 2.2.7 数値ソルバー：実行結果のプロット

In [ ]:
#【初期値問題】数値ソルバーの実行結果の可視化
import pandas as pd
import plotly.express as px

# データフレームの作成
solver_df = pd.DataFrame({
    "Time [s]": sol.t,
    "Velocity [m/s]": sol.y[1],
    "Position [m]": sol.y[0]
})

# プラントと物理オブジェクトの確認
import plotly.express as px
fig = px.line(
    solver_df,
    x="Time [s]",
    y=["Velocity [m/s]", "Position [m]"],
    title="Numerical Solver's Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
)
for trace in fig.data:
    # トレース名（凡例名）が 'Position [m]' であるかを確認
    if trace.name == "Position [m]":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

### 2.2.8 MotionFlowシミュレーションと数値ソルバー解を比較

合格。差異 < 0.001。MotionFlowシミュレーションと数値ソルバーの解は合っている。

In [ ]:
#【初期値問題】MotionFlowの結果と数値ソルバーの結果の比較
print(f"{flow.discrete_time.dt=}[s]")

solver_df["motion_flow_index"] = (solver_df["Time [s]"] / flow.discrete_time.dt).astype(int)
solver_df

In [ ]:
def get_mtnflow_vel_pos_at_time(index: int) -> tuple[float, float]:
    row = mtnflow_df.iloc[index]
    velocity = row["obj_velocity_m_s"]
    position = row["obj_position_m"]
    return velocity, position

solver_df["mtnflow_velocity_m_s"] = solver_df["motion_flow_index"].apply(lambda index: get_mtnflow_vel_pos_at_time(index)[0])
solver_df["mtnflow_position_m"] = solver_df["motion_flow_index"].apply(lambda index: get_mtnflow_vel_pos_at_time(index)[1])

solver_df["vel_diff"] = solver_df["Velocity [m/s]"] - solver_df["mtnflow_velocity_m_s"]
solver_df["pos_diff"] = solver_df["Position [m]"] - solver_df["mtnflow_position_m"]
solver_df

In [ ]:
#【初期値問題】MotionFlowの結果と数値ソルバーの結果の比較結果の可視化
import plotly.express as px
fig_vel = px.line(solver_df, x="Time [s]", y="vel_diff", title="Velocity Difference between Solver and MotionFlow")
fig_vel.show()

In [ ]:
#【初期値問題】MotionFlowの結果と数値ソルバーの結果の比較結果の可視化
import plotly.express as px
fig_pos = px.line(solver_df, x="Time [s]", y="pos_diff", title="Position Difference between Solver and MotionFlow")
fig_pos.show()

# 3. ダンパ係数 = 0 （減衰なし）

合格。
1. 位置、速度の減衰なし。
2. MotionFlowの結果と数値ソルバーの差異 < 0.01

注記：MotionFlowの結果と数値ソルバーの差異が徐々に拡大していく。

In [ ]:
#【ダンパ係数 = 0】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# デフォルトのプロファイル(指令速度0, 指令位置0)をロード
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=0)
# デフォルトのコントローラをロード(推力0)
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=0)
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)
# MDSPhysicalObjectのダンパ係数=0に変更
flow.plant.physical_obj.damper=0.0
# 物理オブジェクトのパラメータを確認
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")
wn_Hz, cc, zeta, wd_Hz = flow.plant.physical_obj.calc_char_values()
pprint.pprint(f"{wn_Hz=} [Hz]")
pprint.pprint(f"{cc=} [Ns/m]")
pprint.pprint(f"{zeta=} [-]")
pprint.pprint(f"{wd_Hz=} [Hz]")
# 物理オブジェクトの初期位置を変える
flow.plant.physical_obj.set_state(acc=0, vel=0, pos=0.1)
# 物理オブジェクトの状態を確認
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")

In [ ]:
#【ダンパ係数 = 0】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【ダンパ係数 = 0】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

In [ ]:
#【ダンパ係数 = 0】数値ソルバーによる解析
import numpy as np
from scipy.integrate import solve_ivp

# パラメータ
m = flow.plant.physical_obj.mass  # 質量 m [kg]
c = flow.plant.physical_obj.damper  # ダンパ係数 c [Ns/m]
k = flow.plant.physical_obj.spring  # ばね定数 k [N/m]
f = 0.0  # 外力0 [N]
print(f"{m=}[kg], {c=}[Ns/m], {k=}[N/m], {f=}[N]")

# 微分方程式の定義
def diff_eq(t, y):
    # 状態ベクトルをアンパック
    x, dot_x = y
    # 位置y0の時間微分 
    dy0_dt = dot_x
    # 速度y1の時間微分 
    dy1_dt = (f - c * dot_x - k * x) / m
    return [dy0_dt, dy1_dt]
# 時間範囲 [s]
t_span = (0, 6)
# 初期状態 [位置 [m], 速度 [m/s]]
x0 = 0.1  # [m]
dot_x0 = 0.0  # [m/s]
y0 = [x0, dot_x0]
# 評価時間点
t_eval = np.arange(t_span[0], t_span[1], 0.1)
# 数値ソルバーの実行
sol = solve_ivp(diff_eq, t_span, y0, t_eval=t_eval)
# 結果の確認
print(f"{sol.success=}")
print(f"{sol.status=}")
print(f"{sol.message=}")
print(f"{sol.t=}")
print(f"{sol.y=}")

In [ ]:
#【ダンパ係数 = 0】数値ソルバーの実行結果の可視化
import pandas as pd
import plotly.express as px

# データフレームの作成
solver_df = pd.DataFrame({
    "Time [s]": sol.t,
    "Velocity [m/s]": sol.y[1],
    "Position [m]": sol.y[0]
})

# プラントと物理オブジェクトの確認
import plotly.express as px
fig = px.line(
    solver_df,
    x="Time [s]",
    y=["Velocity [m/s]", "Position [m]"],
    title="Numerical Solver's Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
)
for trace in fig.data:
    # トレース名（凡例名）が 'Position [m]' であるかを確認
    if trace.name == "Position [m]":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

In [ ]:
#【ダンパ係数 = 0】MotionFlowの結果と数値ソルバーの結果の比較
print(f"{flow.discrete_time.dt=}[s]")

solver_df["motion_flow_index"] = (solver_df["Time [s]"] / flow.discrete_time.dt).astype(int)

def get_mtnflow_vel_pos_at_time(index: int) -> tuple[float, float]:
    row = mtnflow_df.iloc[index]
    velocity = row["obj_velocity_m_s"]
    position = row["obj_position_m"]
    return velocity, position

solver_df["mtnflow_velocity_m_s"] = solver_df["motion_flow_index"].apply(lambda index: get_mtnflow_vel_pos_at_time(index)[0])
solver_df["mtnflow_position_m"] = solver_df["motion_flow_index"].apply(lambda index: get_mtnflow_vel_pos_at_time(index)[1])

solver_df["vel_diff"] = solver_df["Velocity [m/s]"] - solver_df["mtnflow_velocity_m_s"]
solver_df["pos_diff"] = solver_df["Position [m]"] - solver_df["mtnflow_position_m"]
solver_df

In [ ]:
#【ダンパ係数 = 0】MotionFlowの結果と数値ソルバーの結果の比較結果の可視化
import plotly.express as px
fig_vel = px.line(solver_df, x="Time [s]", y="vel_diff", title="Velocity Difference between Solver and MotionFlow")
fig_vel.show()

In [ ]:
#【ダンパ係数 = 0】MotionFlowの結果と数値ソルバーの結果の比較結果の可視化
import plotly.express as px
fig_pos = px.line(solver_df, x="Time [s]", y="pos_diff", title="Position Difference between Solver and MotionFlow")
fig_pos.show()

# 4. ステップ応答
## 4.1 推力ステップ

合格。ばねの伸びの平衡値 = ステップ推力 / ばね定数

In [ ]:
#【推力ステップ】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# デフォルトのプロファイル(指令速度0, 指令位置0)をロード
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=0)
# コントローラをロード (推力ステップ)
step_ctrl_index = 3
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=step_ctrl_index)
pprint.pprint(f"{flow.controller.get_config()=}")
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)
# 物理オブジェクトのパラメータを確認
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")
wn_Hz, cc, zeta, wd_Hz = flow.plant.physical_obj.calc_char_values()
pprint.pprint(f"{wn_Hz=} [Hz]")
pprint.pprint(f"{cc=} [Ns/m]")
pprint.pprint(f"{zeta=} [-]")
pprint.pprint(f"{wd_Hz=} [Hz]")
# 物理オブジェクトの状態を確認
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")

In [ ]:
#【推力ステップ】ばねの伸び
print(f"ステップ推力=2[N]、ばね定数=80[N/m]より、ばねの伸びの平衡値={2/80=} [m]")

In [ ]:
#【推力ステップ】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【推力ステップ】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

## 4.2 位置ステップ

合格。プラント位置：ステップ指令位置に向かって収束する。

In [ ]:
#【位置ステップ】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# プロファイル(位置ステップ)をロード
step_prof_index = 4
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=step_prof_index)
pprint.pprint(type(flow.mprof))
pprint.pprint(flow.mprof.get_config())
# コントローラをロード (PID)
pid_ctrl_index = 1
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=pid_ctrl_index)
pprint.pprint(type(flow.controller))
pprint.pprint(flow.controller.get_config())

In [ ]:
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)
# 物理オブジェクトのパラメータを確認
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")
wn_Hz, cc, zeta, wd_Hz = flow.plant.physical_obj.calc_char_values()
pprint.pprint(f"{wn_Hz=} [Hz]")
pprint.pprint(f"{cc=} [Ns/m]")
pprint.pprint(f"{zeta=} [-]")
pprint.pprint(f"{wd_Hz=} [Hz]")
# 物理オブジェクトの状態を確認
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")

In [ ]:
#【位置ステップ】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【位置ステップ】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
    legend=dict(yanchor="top", y=0.7, xanchor="left", x=0.8)
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

In [ ]:
#【位置ステップ】推力の可視化
import plotly.express as px
fig = px.line(mtnflow_df, x="time_s", y=["force_N"], title="Force")
fig.update_layout(
    xaxis=dict(title="time [s]"),
    yaxis=dict(title="force [N]"),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.7))
fig.show()

# 5. インパルス応答
## 5.1 推力インパルス

合格。速度=インパルス時に最大速度。徐々に減衰。位置=徐々に減衰。

In [ ]:
#【推力インパルス】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# デフォルトのプロファイル(指令速度0, 指令位置0)をロード
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=0)
# コントローラをロード (推力インパルス)
step_ctrl_index = 2
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=step_ctrl_index)
pprint.pprint(f"{flow.controller.get_config()=}")
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)
# 物理オブジェクトのパラメータを確認
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")
wn_Hz, cc, zeta, wd_Hz = flow.plant.physical_obj.calc_char_values()
pprint.pprint(f"{wn_Hz=} [Hz]")
pprint.pprint(f"{cc=} [Ns/m]")
pprint.pprint(f"{zeta=} [-]")
pprint.pprint(f"{wd_Hz=} [Hz]")
# 物理オブジェクトの状態を確認
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")

In [ ]:
#【推力インパルス】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【推力インパルス】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

## 5.2 速度インパルス

合格。速度=インパルス時に最大。位置=徐々に減衰。

In [ ]:
#【速度インパルス】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# プロファイル(推力インパルス)をロード
step_prof_index = 5
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=step_prof_index)
pprint.pprint(type(flow.mprof))
pprint.pprint(flow.mprof.get_config())
# コントローラをロード (PID)
pid_ctrl_index = 1
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=pid_ctrl_index)
pprint.pprint(type(flow.controller))
pprint.pprint(flow.controller.get_config())

In [ ]:
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)
# 物理オブジェクトのパラメータを確認
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")
wn_Hz, cc, zeta, wd_Hz = flow.plant.physical_obj.calc_char_values()
pprint.pprint(f"{wn_Hz=} [Hz]")
pprint.pprint(f"{cc=} [Ns/m]")
pprint.pprint(f"{zeta=} [-]")
pprint.pprint(f"{wd_Hz=} [Hz]")
# 物理オブジェクトの状態を確認
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")

In [ ]:
#【速度インパルス】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【速度インパルス】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
    legend=dict(yanchor="top", y=0.7, xanchor="left", x=0.8)
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

# 6. 摩擦ありの場合

## 6.1 ステップ推力 < 静止摩擦力

合格。物理オブジェクトは動かない。

In [ ]:
#【推力ステップ（摩擦あり）（ステップ推力 < 静止摩擦力）】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# デフォルトのプロファイル(指令速度0, 指令位置0)をロード
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=0)
# コントローラをロード (推力ステップ)
step_ctrl_index = 3
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=step_ctrl_index)
pprint.pprint(f"{flow.controller.get_config()=}")
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)

In [ ]:
# 摩擦係数を変更
flow.plant.physical_obj.static_friction_coeff=0.2
flow.plant.physical_obj.dynamic_friction_coeff=0.05
print(f"{flow.plant.physical_obj.static_friction_coeff * 9.80665 * flow.plant.physical_obj.mass=} [N] (静止摩擦力)")
print(f"{flow.plant.physical_obj.dynamic_friction_coeff * 9.80665 * flow.plant.physical_obj.mass=} [N] (動摩擦力)")

In [ ]:
#【推力ステップ（摩擦あり）（ステップ推力 < 静止摩擦力）】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【推力ステップ（摩擦あり）（ステップ推力 < 静止摩擦力）】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
    legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.8)
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

## 6.2 摩擦あり（ステップ推力 > 静止摩擦力）

In [ ]:
#【推力ステップ（摩擦あり）（ステップ推力 > 静止摩擦力）】MotionFlow作成
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# デフォルトのプロファイル(指令速度0, 指令位置0)をロード
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=0)
# コントローラをロード (推力ステップ)
step_ctrl_index = 3
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=step_ctrl_index)
pprint.pprint(f"{flow.controller.get_config()=}")
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)

In [ ]:
# 摩擦係数を変更
flow.plant.physical_obj.static_friction_coeff=0.05
flow.plant.physical_obj.dynamic_friction_coeff=0.01
print(f"{flow.plant.physical_obj.static_friction_coeff * 9.80665 * flow.plant.physical_obj.mass=} [N] (静止摩擦力)")
print(f"{flow.plant.physical_obj.dynamic_friction_coeff * 9.80665 * flow.plant.physical_obj.mass=} [N] (動摩擦力)")

In [ ]:
#【推力ステップ（摩擦あり）（ステップ推力 > 静止摩擦力）】MotionFlow実行
mtnflow_df = flow.execute()
mtnflow_df

In [ ]:
#【推力ステップ（摩擦あり）（ステップ推力 > 静止摩擦力）】MotionFlow実行結果の可視化
import plotly.express as px
fig = px.line(
    mtnflow_df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
    legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.8)
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

In [ ]:
ext_force = 2  # [N] 外力
k = 80  # [N/m] ばね定数
sfric_force = flow.plant.physical_obj.static_friction_coeff * 9.80665 * flow.plant.physical_obj.mass  # [N] 静止摩擦力
dfric_force = flow.plant.physical_obj.dynamic_friction_coeff * 9.80665 * flow.plant.physical_obj.mass  # [N] 静止摩擦力
print(f"{ext_force=} [N]")
print(f"{k=} [N/m]")
print(f"{sfric_force=} [N]")
print(f"{dfric_force=} [N]")

print(f"{ext_force/k=} [m]")
print(f"{(ext_force - sfric_force)/k=} [m]")
print(f"{(ext_force - dfric_force)/k=} [m]")